# Замер производительности SomeSAT

В данной среде представлен замер производительности реализации алгоритма DPLL. 

[Репозиторий с реализацией алгоритма на языке F#](https://github.com/artem-burashnikov/SomeSAT).

Последующие разделы включают характеристики используемого оборудования и способ подготовки тестового стенда, описание набора тестовых данных, используемые метрики, исследовательские гипотезы и анализ полученных результатов.

---

## Характеристики используемого оборудования

Ниже приведены основные характеристики и команды `bash` для их вывода.

### Использованные команды

`cat /etc/os-release` &mdash; информация об операционной системе.

`lscpu` &mdash; информация об используемом процессоре.

`free -h` &mdash; информация об используемой памяти.

### Характеристики тествого стенда

1. **OS**: Ubuntu 22.04.4 LTS

2. **CPU vendor and model**: AMD Ryzen 5 4500U

3. **Architecture**: x86-64

4. **CPU frequency (MAX)**: 2.38 GHz

5. **RAM (Total)**: 7.1Gi

Измерения проводились на одном ядре на одном потоке.

---

## Подготовка тестового стенда

Часть команд выполнена с помощью утилиты `cpupower`, которая может быть установлена с помощью:

```sh
sudo apt install linux-tools-common
```

Все изменения временные и вернутся в состояние по умолчанию при перезагрузке системы:

1. Отключение `swap`:

```sh
sudo swapoff -a
```

2. Установка подсистемы `CPUFreq governor` в состояние `performance`:

```sh
sudo cpupower frequency-set -g performance
```

3. Установка максимального значения для минимальной и максимальной частоты процессора для конкретного ядра (в данном случае с порядковым номером 0):

```sh
sudo cpupower frequency-set -d 2.38GHz -u 2.38GHz -c 0
```

3. Сборка программы в Release (запустить в корне проекта):

```sh
dotnet build -c Release
```

3. Запуск исполняемой программы на одном ядре (в данном случае с порядковым номером 0, запустить в корне собранного проекта):

```sh
taskset -c 0 ./SomeSAT/bin/Release/net8.0/SomeSAT <file_path>
```

---

## Использованная метрика

Измерено чистое время работы алгоритма (то есть время работы программы за исключением времени, затраченного на операцию чтения данных из файла и операцию перевода прочитанных данных во внутреннее представление и обратно).

---

## Тестовые данные

Для измерения времени выполенения выбраны данные, время работы алгоритма на которых составляет порядка 4-5 секунд.

Тестовые данные размещены в директории `test_data/`, расположенной в корне текущего репозитория.

Файл `jnh301.cnf` содержит 100 переменных и 900 дизъюнктов. Булева формула, представленная в файле, является разрешимой.

---

## Инструмент для измерения

Для измерения времени использованы методы класса `Stopwatch`, предоставляемого `.NET`.

---

## Исследовательская гипотеза

Ожидается, что полученный набор измерений соответствует нормальному распределению.

---

## Проведение измерений

Для проведения измерений соблюдены следующие условия:

1. Перед исполнением алгоритма вызывается `Thread.Sleep(10000)` для искусственного простоя программы в режиме ожидания.

2. Выполняется предварительный прогрев `.NET` путем неизмеряемого исполнения программы на `warmup_data.cnf`, после чего алгоритм выполняется на тестовых данных с замером времени исполнения.

## Список литературы

1: https://temci.readthedocs.io/en/latest/temci_exec.html

2: https://gernot-heiser.org/benchmarking-crimes.html